In [1]:
import pandas as pd
import requests
import warnings
import time
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor
from ipapy import is_valid_ipa
# from google.colab import files
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [2]:
main_csv="ebanglalibrary_100K.csv"


main_csv_validated= main_csv[0:-4]+"_validated"+main_csv[-4:]
main_csv_validated_with_words= main_csv[0:-4]+"_validated_with_words"+main_csv[-4:]

print(main_csv)
print(main_csv_validated)
print(main_csv_validated_with_words)

ebanglalibrary_100K.csv
ebanglalibrary_100K_validated.csv
ebanglalibrary_100K_validated_with_words.csv


In [4]:
df = pd.read_csv(main_csv)
# df_words=pd.read_csv("ipa_validation_batch1.xlsx - Sheet1.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'ebanglalibrary_100K.csv'

In [7]:
def replace_func(subset):
  for row in subset.itertuples():
    ipa=row.ipa+"%"
    ipa=ipa.replace("a","ɐ")
    ipa=ipa.replace("i","ɪ")
    ipa=ipa.replace("æ","ɛ")
    ipa=ipa.replace("r","ɾ")
    ipa=ipa.replace("ʲ ","e̯ ")
    for i, ch in enumerate(ipa):
      if ch=='ʲ':
        if ipa[i+1]==' ':
          ipa = ipa[:i] + "e̯" + ipa[i+1:]
        elif is_valid_ipa(ipa[i+1])==False:
          ipa = ipa[:i] + "e̯" + ipa[i+1:]

    ipa=ipa[:-1]
    df.at[row.Index,"validated_ipa"]=ipa

    

    
    # words=row.sentence.split(" ")
    # for word in words:
    #   if word[-1]=='ʲ':
    #     word=word[:-1] + "e̯"
        
    #   for ch in word:
        

In [8]:
df=pd.read_csv(main_csv)
start=0
steps=1000
end=df.shape[0]
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(replace_func, subset))
print(pool.shutdown())
df.to_csv(main_csv_validated,index=False)

106493
None


In [8]:
prefix_sentence="__$$__"
prefix_words="__##__"

In [4]:
prefix_sentence="__$$__"
prefix_words="__##__"
def replace_words(subset):
  all_sentences=""

  # combine all sentence from subset and replace all words from word list with special characters
  for row in subset.itertuples():
    if pd.notna(row.done):
      continue
    if all_sentences=="":
      all_sentences=row.sentence
    else:
      all_sentences=all_sentences+prefix_sentence+row.sentence
  for row in df_words.itertuples():
    if pd.isna(row.ɪpas) or pd.isna(row.words):
      continue
    if pd.notna(row.word_fix):
      all_sentences=all_sentences.replace(" "+row.words+" "," "+prefix_words+row.word_fix+" ")
      all_sentences=all_sentences.replace(prefix_sentence+row.words+" ",prefix_sentence+prefix_words+row.word_fix+" ")
      all_sentences=all_sentences.replace(" "+row.words+"।"," "+prefix_words+row.word_fix+"।")
      all_sentences=all_sentences.replace(" "+row.words+"?"," "+prefix_words+row.word_fix+"?")
      all_sentences=all_sentences.replace(" "+row.words+"!"," "+prefix_words+row.word_fix+"!")


    all_sentences=all_sentences.replace(" "+row.words+" "," "+prefix_words+row.ɪpas+" ")
    all_sentences=all_sentences.replace(prefix_sentence+row.words+" ",prefix_sentence+prefix_words+row.ɪpas+" ")
    all_sentences=all_sentences.replace(" "+row.words+"।"," "+prefix_words+row.ɪpas+"।")
    all_sentences=all_sentences.replace(" "+row.words+"?"," "+prefix_words+row.ɪpas+"?")
    all_sentences=all_sentences.replace(" "+row.words+"!"," "+prefix_words+row.ɪpas+"!")
    all_sentences=all_sentences.replace(" "+row.words+","," "+prefix_words+row.ɪpas+",")
    all_sentences=all_sentences.replace(" "+row.words+":"," "+prefix_words+row.ɪpas+":")
    all_sentences=all_sentences.replace(" "+row.words+";"," "+prefix_words+row.ɪpas+";")
    all_sentences=all_sentences.replace(" "+row.words+"’"," "+prefix_words+row.ɪpas+"’")
    all_sentences=all_sentences.replace(" "+row.words+"”"," "+prefix_words+row.ɪpas+"”")
    all_sentences=all_sentences.replace("“"+row.words+" ","“"+prefix_words+row.ɪpas+" ")
    all_sentences=all_sentences.replace("("+row.words+" ","("+prefix_words+row.ɪpas+" ")
    all_sentences=all_sentences.replace("("+row.words+",","("+prefix_words+row.ɪpas+",")
    all_sentences=all_sentences.replace(" "+row.words+")"," "+prefix_words+row.ɪpas+")")
    
    
  all_sentences_arr=all_sentences.split(prefix_sentence);

  for row,sentence_v in zip(subset.itertuples(),all_sentences_arr):
    if pd.notna(row.done):
      continue

    validated_ipa=row.validated_ipa.split(" ")
    new_ipa=sentence_v.split(" ")
    new_ipa_sentence=""
    if new_ipa[0]=="":
      new_ipa=new_ipa[1:]
    if validated_ipa[0]=="":
      validated_ipa=validated_ipa[1:]
    for word,valid in zip(validated_ipa, new_ipa):
      if valid.startswith(prefix_words):
        new_ipa_sentence=new_ipa_sentence+valid+" "
        # new_ipa_sentence=new_ipa_sentence+valid.replace(prefix_words,"")+" "
      else:
        new_ipa_sentence=new_ipa_sentence+word+" "
    
    
    
    # .replace(prefix_words,"")

    # print(row.sentence)
    # print(row.ipa)
    # print(new_ipa_sentence)
    # print("++++++++++++++++++++++++++++++++++++++")
    df.at[row.Index,"validated_ipa"]=new_ipa_sentence
    df.at[row.Index,"done"]="OK"
      
    # df.at[row.Index,"temp_sentence"]=all_sentences_arr[row.Index+1]

    

In [5]:
# threadpool make sure everything is OK before running this cell
df=pd.read_csv(main_csv_validated)
try:
  df=pd.read_csv(main_csv_validated_with_words)
except:
  print(f"{main_csv_validated_with_words} file found.\nUsing {main_csv_validated} file to create {main_csv_validated_with_words} file")
start=0
steps=20
end=106500
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(replace_words, subset))
print(pool.shutdown())
df.to_csv(main_csv_validated_with_words,index=False)


ebanglalibrary_100K_validated_with_words.csv file found.
Using ebanglalibrary_100K_validated.csv file to create ebanglalibrary_100K_validated_with_words.csv file
None


In [9]:
# clean all prefix_words
df = pd.read_csv("ebanglalibrary_100K_validated_with_words_bn_mark.csv")
for row in df.itertuples():
  df.at[row.Index,"clean_validated_ipa"]=row.validated_ipa.replace(prefix_words,"")
df.to_csv("ebanglalibrary_100K_validated_with_words_bn_mark.csv",index=False)

In [6]:
# remove done column
df = pd.read_csv(main_csv_validated_with_words)
df=df.drop(columns=['done'])
df.to_csv(main_csv_validated_with_words,index=False)

In [6]:
# set ranged done to nan
df = pd.read_csv(main_csv_validated_with_words)
sub=df.iloc[18000:20000] #set the range
for row in sub.itertuples():
  df.at[row.Index,"done"]=None
df.to_csv(main_csv_validated_with_words,index=False)

In [ ]:
#       __##__(.)*



END of the SCRIPT


In [9]:
# test cell
df = pd.read_csv(main_csv_validated_with_words)
df_words=pd.read_csv("ipa_validation_batch1.xlsx - Sheet1.csv")
replace_words(df.iloc[0:100])

In [299]:
# sort df_words by length of word
df=pd.read_csv("ipa_validation_batch1.xlsx - Sheet1.csv")
def length(word):
    if type(word) == str:
        return len(word)
    return 0
df_words['length'] = (df_words['words']).apply(length)
df_words = df_words.sort_values(by=['length'])
df_words = df_words.reset_index(drop=True)
df_words = df_words.drop(columns=['length'])
df_words = df_words.drop_duplicates(subset=['words'], keep='first')
df_words.to_csv('cleanWords.csv', index=False)

In [ ]:
for row in df.itertuples():
  if pd.isna(row.done):
    print(f"{row.Index} {row.sentence}")
  if row.Index>10001:
    break

In [249]:
df.to_csv(f"ebanglalibrary_100k_with_ipa0_106500_validated_{start_pos}_{end}.csv",index=False)

In [23]:
name="sourav ahmed"
# name[-1]="x"
name=name[:-1] + "x"
print(name.replace("a","A"))
print(name)

sourAv Ahmex
sourav ahmex


In [109]:
unique_char_bangla=[' ', '!', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '£', 'ª', '\xad', '¯', '·', 'Æ', 'â', '˃', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', 'ৱ', '৷', '৺', 'ਆ', '\u200c', '\u200d', '‒', '–', '—', '‘', '’', '“', '”', '•', '…', '−', '√', '∣', '⋯', '\uf020', '\uf02c', '\uf02d', '\uf05b', '\ufeff']
unique_char_ipa=[' ', '!', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '£', '\xad', 'æ', 'ŋ', 'ɔ', 'ɟ', 'ɽ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', '̃', '̪', '̯', '͡', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', '৷', '–', '—', '‘', '’', '“', '”', '•', '…', '∣', '\ufeff']

# for row in df.itertuples():
#   for ch in row.ipa:
#     if ch not in unique_char_ipa:
#       unique_char_ipa.append(ch)
# unique_char_ipa.sort()
# print(unique_char_ipa)

In [163]:
print(" ".join(unique_char_ipa))

  ! % ' ( ) * , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? A B C D E F G H I J K L M N O P Q R S T U V W Y [ ] _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { | ~   £ ­ æ ŋ ɔ ɟ ɽ ʃ ʰ ʱ ʲ ʷ ̃ ̪ ̯ ͡ ا ث خ د ر س ص ع ق ل ه ي । ॥ ৷ – — ‘ ’ “ ” • … ∣ ﻿


MISC


In [ ]:
      # if ch=='ʲ' and (row.sentence[i+1]==" " or row.sentence[i+1]=="\n" or  row.sentence[i+1]=="।" or row.sentence[i+1]=="?" or row.sentence[i+1]=="!" or row.sentence[i+1]=="%" or ):


In [111]:
# for item1, item2, item3 in zip([1, 2, 3,], ['a', 'b', 'c','d'], [True, False, True,'d']):
#     print(item1, item2, item3)



    # print(row.sentence)
    # print(row.ipa)
    # print(new_ipa_sentence)
    # print("++++++++++++++++++++++++++++++++++++++")
    

1 a True
2 b False
3 c True


In [194]:
unique_words_test=[]
sub=df.iloc[2884:2994]
for row in sub.itertuples():
  for word in row.sentence.split(" "):
    if word not in unique_words_test:
      unique_words_test.append(word)


unique_words_test

['ধোয়াটের',
 'জলে',
 'ভেসে-আসা',
 'ভরাটের',
 'মাটি',
 'নয়,',
 'সে',
 'একটি',
 'নতুন',
 'নিঃসঙ্গ',
 'নদী।সিটি',
 'কলেজে',
 'লেকচারারের',
 'কাজ',
 'করত',
 'জীবনানন্দ।',
 'আর',
 'ভালো',
 'হয়ে',
 'উঠেই',
 'আবার',
 'লেগেছি',
 'জীবিকার্জনের',
 'সংগ্রামে।এ',
 'রুগীর',
 'পক্ষে',
 'দুমকার',
 'পথ',
 'তো',
 'সাধ্যাতীত।',
 'সেই',
 'অল্পজলের',
 'প্রেমে',
 'সর্বস্বান্ত',
 'হয়েছিল',
 'কখনো?',
 'যেন',
 'এই',
 'সংগ্রামসংকুল',
 'সংসারের',
 'জন্যে',
 'সংসারপলাতক।',
 'সময়',
 'নেই,',
 'লাস্ট',
 'ট্রাম',
 'চলে',
 'গেল',
 'বোধ',
 'হয়।পরদিন',
 'গল্পের',
 'বাকিটা',
 'আমরাও',
 'শুনতে',
 'পেলাম।',
 'ওটা',
 'তখনো',
 'যুগ',
 'যে-যুগে',
 'প্রায়',
 'প্রেমেরই',
 'সমান-সমান',
 'বন্ধুতার',
 'দাম',
 'ছিল—সেই',
 'একই',
 'বিরহোৎকণ্ঠ',
 'বন্ধুতা।',
 'তার',
 'আগে',
 'সাবিত্রীপ্রসন্নের',
 'আমলেই',
 'নৃপেন',
 'বিজলীতে',
 'নাট্যসমালোচনা',
 'লিখত।',
 'ইডেন',
 'হিন্দু',
 'হসটেলের',
 'বাথরুমে',
 'দরজা',
 'বন্ধ',
 'করে',
 'কার্বলিক',
 'এসিড',
 'খেয়ে',
 'কথক',
 'বিজয়',
 'সেনগুপ্ত',
 'আত্মহত্যা',
 'করেছে।দেখতে',
 'গিয়েছিলাম',
 'তাকে

In [195]:
for row in df_words.itertuples():
  if row.words in unique_words_test:
    print(row.words)

এর
বাড়িতে
অকালে
অটুট
অতীতে
অত্যধিক
অনেকখানি
অন্ত
অন্তরালে
অন্য
অর্থাৎ
আইনে
আকাশের
আগে
আছে
আজ
আজও
আত্মহত্যা
আত্মার
আত্মিক
আধুনিক
আনন্দিত
আনেন
আপনাদের
আপনার
আবার
আমরা
আমরাও
আমাকে
আমাদের
আমার
আমি
আমিও
আর
আরো
আলাপ
আলো
আশা
আশ্বাস
আসবে
আসল
আসে
ইডেন
উচিত
উচ্চ
উঠতে
উড়িয়ে
উত্তর
উৎস
উদীয়মান
উদ্দেশ্য
উদ্ভূত
উপন্যাস
উপর
উপস্থিত
উপায়
উপেক্ষা
এ
এই
এক
একই
একজন
একটা
একটি
একটিও
একদিন
একবার
একমাত্র
একসময়
এটি
এতগুলো
এমন
এমনটি
এমনি
এর
এরই
এল
এলেন
এসিড
এসে
এসেও
ঐক্য
ও
ওকে
ওটা
ওদের
ওর
ক
কখন
কখনো
কটা
কতটা
কতটুকুইবা
কথা
কথার
কপি
কবি
কবিতা
কবিতায়
কবে
করত
করতাম
করতে
করতেও
করল
করলাম
করলেন
করা
করুন
করে
করেই
কলকাতায়
কলেজে
কল্পনাও
কল্পনার
কল্লোল
কল্লোলের
কাউকে
কাউকেই
কাজ
কারণ
কি
কিনা
কিন্তু
কী
কে
কেউ
কেন
কেমন
কোঠায়
কোন
কোনো
কোলাকুলি
কৌশলে
ক্যামেরা
ক্লিক
ক্ষীণ
ক্ষেতের
ক্ষেত্রে
খাইয়ে
খাঁটি
খুঁজে
খুলে
খুশি
খেয়ে
গভীর
গর্ব
গলা
গল্প
গল্পের
গা
গিয়ে
গিয়েছিল
গিয়েছিলাম
গুরুবন্দনা
গেল
গোটা
গোল
গোলে
গ্রীষ্মের
ঘর
ঘরে
চক্ষু
চমৎকার
চলে
চারপাশের
চালিয়েছিলেন
চিঠি
চিঠির
চিনতে
চিনির
চুপ
চেয়ে
চেষ্টায়
চোখে
ছড়া
ছবিটি
ছবিটির
ছাত্রকে
ছাত্

In [1]:
for i, ch in enumerate(" t̪ɪnɪ ɟɐt̪ɪbʱed̪ mɐnɪt̪en nɐ, hɪnd̪u muʃɔlmɐnke ʃɔmobʱɐbe d̪ekʰɪt̪en o ʃɪʃʃod̪ɪgeɾ mod̪d̪ʱe hɪnd̪u-muʃɔlmɐn ʃɔkol ɟɐt̪ɪke͡ɪ̯ gɾohon koɾɪt̪en।"):
  print(f"{i}:{ch}")

0: 
1:t
2:̪
3:ɪ
4:n
5:ɪ
6: 
7:ɟ
8:ɐ
9:t
10:̪
11:ɪ
12:b
13:ʱ
14:e
15:d
16:̪
17: 
18:m
19:ɐ
20:n
21:ɪ
22:t
23:̪
24:e
25:n
26: 
27:n
28:ɐ
29:,
30: 
31:h
32:ɪ
33:n
34:d
35:̪
36:u
37: 
38:m
39:u
40:ʃ
41:ɔ
42:l
43:m
44:ɐ
45:n
46:k
47:e
48: 
49:ʃ
50:ɔ
51:m
52:o
53:b
54:ʱ
55:ɐ
56:b
57:e
58: 
59:d
60:̪
61:e
62:k
63:ʰ
64:ɪ
65:t
66:̪
67:e
68:n
69: 
70:o
71: 
72:ʃ
73:ɪ
74:ʃ
75:ʃ
76:o
77:d
78:̪
79:ɪ
80:g
81:e
82:ɾ
83: 
84:m
85:o
86:d
87:̪
88:d
89:̪
90:ʱ
91:e
92: 
93:h
94:ɪ
95:n
96:d
97:̪
98:u
99:-
100:m
101:u
102:ʃ
103:ɔ
104:l
105:m
106:ɐ
107:n
108: 
109:ʃ
110:ɔ
111:k
112:o
113:l
114: 
115:ɟ
116:ɐ
117:t
118:̪
119:ɪ
120:k
121:e
122:͡
123:ɪ
124:̯
125: 
126:g
127:ɾ
128:o
129:h
130:o
131:n
132: 
133:k
134:o
135:ɾ
136:ɪ
137:t
138:̪
139:e
140:n
141:।


In [14]:
df=pd.read_csv(main_csv_validated_with_words);
punctuations= [',','?','!',"।",")","(","“","”","'"]
bangla_ch=['ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', 'ৰ', 'ৱ', '৷', '৺']

print(len(bangla_ch))
sub=df.iloc[0:100]
for row in sub.itertuples():

  words=row.clean_validated_ipa.split(" ")
  for word in words:
    for ch in punctuations:
      if ch in word:
        if word[-1]==ch  or word[0]==ch:
          break
        else:
          print(f"{word}>>>>>>>>>>>> {row.clean_validated_ipa}")
          break

67
nɔhe।এখন>>>>>>>>>>>> ʃeɪ̯ ɟonno bɐŋgɐlɐɾ ɪt̪ɪhɐʃ ɐɾɟo͡ɐ̯bɔɾt̪eɾ ɪt̪ɪhɐʃeɾ ɔŋʃomɐt̪ɾo–t̪ɐhɐ hoɪ̯t̪e ʃɔt̪ɔnt̪ɾo bɐ pɾɪt̪ʰɔk nɔhe।এখন ʃɔmogɾo ɐɾɟɐbɔɾt̪eɾ od̪ʱɪbɐʃɪ ɔd̪ʱoppot̪ɪt̪o bolɪʲɐ poɾɪcɪt̪o। 
nɐmɐnt̪ɔɾmɐt̪ɾo।kot̪ɪ>>>>>>>>>>>> ʃut̪ɔɾɐŋ ot̪ɪt̪ke buɟʱɪbɐɾ ceʃtɐ bɔɾt̪t̪omɐnke buɟʱɐ͡ɪ̯bɐɾ ceʃtɐɾ nɐmɐnt̪ɔɾmɐt̪ɾo।kot̪ɪ ɔlpod̪ɪnmɐt̪ɾo ɐmɐd̪eɾ d̪eʃe eɪ̯ ʃubʱo ceʃtɐɾ ʃut̪ɾopɐt̪ ho͡ɪ̯ʲɐcʰe। 
koɾɪʲɐcʰɪlo।ɟɐ̃hɐɾɐ>>>>>>>>>>>> e͡ɪ̯ɾupe pou̯ɾɐnɪk ɐkkʰɐʲɪkɐ kɾome nɐnɐ ɐkɐɾ d̪ʱɐɾon koɾɪʲɐcʰɪlo।ɟɐ̃hɐɾɐ ɐɾɟɐbɔɾt̪eɾ bɐhɪɾe ɐɾɟo-pɾobʱɐb kʰet̪ɾo pɾot̪ɪʃtʱɪt̪o koɾɪʲɐcʰɪlen, t̪ɐ̃hɐɾɐ ɐɾɟo nɐmeɪ̯ poɾɪcɪt̪o cʰɪlen। 
cɐhe,–ɐʲɐs>>>>>>>>>>>> t̪ɐhɐɾɐ ɐloʃʃo cɐhe,–ɐʲɐs ʃɪkɐɾ koɾɪt̪e ɔʃɔmmɔt̪ɔ। 
kɔt̪ʰɐ।তাহার>>>>>>>>>>>> ʃut̪ɔɾɐŋ ɪhɐ bɐŋgɐlɪɾ ɪt̪ɪhɐʃeɾ ʃɔɾbɐpekkʰɐ ullekʰɟoggo kɔt̪ʰɐ।তাহার mulomɔnt̪ɾo ɛkot̪ɐ,–তাহার mulomɔnt̪ɾo ʃɐɾt̪ʰot̪t̪ɐgɔ–t̪ɐhɐɾ mulomɔnt̪ɾo ɔkɾɪt̪ɾɪm ɔnɐbɪl ɔpɐɾ ʃɔd̪eʃpɾɪt̪ɪ। 
ɛkot̪ɐ,–তাহার>>>>>>>>>>>> ʃut̪ɔɾɐŋ ɪhɐ bɐŋgɐlɪɾ ɪt̪ɪhɐʃeɾ ʃɔɾbɐpekkʰɐ ullekʰɟoggo kɔt̪ʰɐ।তাহার mulomɔnt̪ɾo ɛkot̪ɐ,–তাহা

In [87]:
bangla_ch=['ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', 'ৱ', '৺']

bangla_ch.sort()

print(bangla_ch)

['ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', 'ৱ', '৺']


In [7]:
df=pd.read_csv("100k_removed_num_and_bangla.csv");
df.drop(columns=['bn_mark'],inplace=True)
bangla_ch=['ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', 'ৰ', 'ৱ', '৺','0','1','2','3','4','5','6','7','8','9','০','১','২','৩','৪','৫','৬','৭','৮','৯']
sub=df.iloc[0:106500]
indexes=[]
for row in sub.itertuples():
  words=row.validated_ipa.split(" ")
  for word in words:
    flag=False
    for ch in bangla_ch:
      if ch in word:
        indexes.append(row.Index)
        # remove row
        # df.at[row.Index,"bn_mark"]="ERROR"
        df.drop(row.Index, inplace=True)
        flag=True
        break
    if flag:
      break

print(len(indexes))
print(len(set(indexes)))
# sort df by bn_mark
# df=df.sort_values(by=['bn_mark'])
df.to_csv("100k_removed_num_and_bangla.csv",index=False)

4
4


In [10]:
df=pd.read_csv("100k_removed_num_and_bangla.csv");
df.drop(columns=['ipa'],inplace=True)
df.drop(columns=['validated_ipa'],inplace=True)
df.to_csv("100k_removed_num_and_bangla.csv",index=False)

In [23]:
print(len(set(indexes)))

3767


In [66]:
df=df.sort_values(by=['bn_mark'])

df.to_csv("ebanglalibrary_100K_validated_with_words_bn_mark.csv",index=False)

In [79]:
df=pd.read_csv("ebanglalibrary_100K_validated_with_words_bn_mark.csv");
df.drop(columns=['bn_mark'],inplace=True)
df.to_csv("ebanglalibrary_100K_validated_with_words_bn_mark.csv",index=False)

In [68]:
df=pd.read_csv("ebanglalibrary_100K_validated_with_words_bn_mark.csv");
df_new=pd.read_csv("error_ipa.csv")
punctuations= [',','?','!',"।",")","(","“","”","'"]
bangla_ch=['ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', 'ৱ', '৺']

sub=df.iloc[0:106500]
indexes=[]

for row in sub.itertuples():
  for ch in bangla_ch:
    if ch in row.validated_ipa:
      # append to df_new
      df_new.loc[len(df_new)]=[row.Index,row.sentence,row.validated_ipa]
      indexes.append(row.Index)

      break
print(len(indexes))
# sort df by bn_mark

3477


In [30]:
df_new.to_csv("error_ipa.csv",index=False)

In [31]:
bn_mark_csv="trainIPAdb_validated_with_words_bn_mark.csv"
df=pd.read_csv(bn_mark_csv);
df_words=pd.read_csv("ipa_validation_batch1.xlsx - Sheet1.csv")
df_error=pd.read_csv("error_ipa2_50k.csv")
sub=df.iloc[0:106500]
bangla_ch=['ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', 'ৰ', 'ৱ', '৺']
unique_char_ipa_v2=[' ', '!', '#', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '£', '\xad', '·', 'ŋ', 'ɐ', 'ɔ', 'ɛ', 'ɟ', 'ɪ', 'ɽ', 'ɾ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', 'ː', '̃', '̆', '̪', '̯', '͡', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', '৷', '\u200c', '–', '—', '‖', '‘', '’', '“', '”', '•', '…', '∣', '\ufeff']
punctuations= [',','?','!',"।",")","(","“","”","'"," ",'-',' ', '!', '#', '%', "'", '(', ')', '*', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?','{', '|','৷', '–', '—', '‖', '‘', '’', '“', '”', '•', '…', '∣']
not_validated_words=[]
for row in sub.itertuples():
  words=row.validated_ipa.split(" ")
  ipas=row.ipa.split(" ")
  if words[0]=="":
    words=words[1:]
  if ipas[0]=="":
    ipas=ipas[1:]
  for word,ipa in zip(words,ipas):
    bn_word=""
    ipa_ipa=""
    for ch in word+"#":
      if ch in punctuations or ch==" " or ch=="#":
        if bn_word!="":
          print(bn_word)
          valid=df_words.loc[df_words['words'] == bn_word]
          if valid.empty:
            if bn_word not in not_validated_words:
              not_validated_words.append(bn_word)
              df_error.loc[len(df_error)]=[row.Index,bn_word,ipa,ipa,row.text,row.validated_ipa]
          else:
            # print(valid["words"])
            print(f"+++++++++++{valid['words'].values[0]} : {valid['ɪpas'].values[0]}")
            df.at[row.Index,"validated_ipa"]=row.validated_ipa.replace(bn_word,valid['ɪpas'].values[0])
          bn_word=""
          ipa_ipa=""
      elif ch in bangla_ch:

        bn_word=bn_word+ch

df.to_csv(bn_mark_csv,index=False)
df_error.to_csv("error_ipa2_50k.csv",index=False)
    

আন্তরিকতার
নেভায়নি
বেদনার
ক্ষুণ্ণ
মোসাম্মাৎ
ক্ষুণ্ণ
কেমিকাল
শ্রেণিতে
ঘুড়ির
শ্রীলঙ্কার
সাউন্ডপ্রুফ
শ্রীলঙ্কায়
শ্রীলঙ্কার
ক্ষুণ্ণ
ক্ষুণ্ণ
শ্রেণিতে
গ্যাস্টোলজি
র্যালি
বিদ্রুপ
শ্রীলঙ্কার
রত্নায়েক
লোহাগাড়ার
জন্মদিনে
+++++++++++জন্মদিনে : ɟɔnmod̪ɪne
গাণিতিক
ক্ষুণ্ণ
মোগলা
সাতভাইয়া
কেমিকাল
পেসাররাই
শ্রীলঙ্কার
কেমিকাল
শ্রীলঙ্কাকে
কেমিকালে
রেডডেভিলস
পিডব্লুডি
পিডব্লুডি
রান্নাবান্নাসহ
আইয়ুবর
শ্রেণিকক্ষের
পল্টু
ক্ষুণ্ণ
শীৎকার
শ্রীলঙ্কাতেও
আত্মহত্যায়
ক্যারোলিনার
খোশমেজাজে
ক্ষুণ্ণ
শ্রেণিতে
রাজনৈতিকদের
শ্রীলঙ্কার
পোষেন
মেইডেন
ক্ষুণ্ণ
রেকোর্ডিংয়ের
শত্রুজিৎপুর
পরম্পরা
রাওনিকের
চট্টলাবাসীর
শ্রেণিতে
শ্রেণিতে
শ্রেণিকক্ষ
প্রত্যঙ্গ
শ্রেণিভেদে
ল্যাকটেটিং
গুনারন্তে
সবাই
+++++++++++সবাই : ʃɔbɐɪ
পাড়ুকোনের
কর্তা
গ্র্যাজুয়েটদের
মেন্দা
সখিপুরে
সখিপুরে
শিক্ষাপ্রতিষ্ঠানটির
পুরোহিতদের
লোহাগাড়ার
শ্রীলঙ্কার
মৃত্যুকেই
পিছনের
িহ্নত
শ্রেণিতে
ক্ষুণ্ণকারীদের
পেট্রোলিংয়ের
দ্রিস
ছে


In [90]:
# not_validated_words array to dataframe
# df_not_validated_words=pd.DataFrame(not_validated_words, columns=['words'])
# df_not_validated_words.to_csv("not_validated_words.csv",index=False)
not_validated_words.sort()
print(not_validated_words)

['অধঃপতন', 'অবনীনাথ', 'অমরনাথবাবু', 'অ্যাই', 'আচ্ছা', 'আজ্ঞে', 'আয়', 'আলবাৎ', 'ইদানিং', 'ইয়ে', 'ঈশানী', 'উঁহু', 'ঊনিশ', 'এ্যাঁ', 'ওঁরে', 'ওঃ', 'ওস্তাদ', 'ওহ্', 'কই', 'করেছিলো', 'কর্তা', 'কালকা', 'কুমারবাবু', 'খন', 'গাড়িতে', 'গিন্নী', 'গুড়', 'গ্লোরিয়া', 'ঘংঘাড়াম', 'চল', 'চ্যাং', 'ছি', 'জয়ন্ত', 'জানাশোনা', 'জিনা', 'জী', 'ঠান্ডা', 'ড', 'ডাক্তারবাবু', 'ত', 'তাকাও', 'তাপর', 'তুর্কী', 'দস্যুসম্রাট', 'দাদুন', 'দু', 'দেখ', 'দেখোনি', 'ধেৎ', 'নচ্ছার', 'নয়ন', 'নাগরাজন', 'নাহ', 'নিবৃত্তি', 'নিশ্চয়', 'নে', 'নো', 'পনেরো', 'পশ্চিমবঙ্গ', 'পাঠান্তর', 'পালাল', 'পুঁটু', 'প্রাণবতী', 'ফুঃ', 'ব', 'বরাবরেষু', 'বর্ধমানে', 'বলিয়ো', 'বসো', 'বহেন', 'বাবারে', 'বাসরে', 'বিপিন', 'বুঝলি', 'বেচারা', 'বেন্থাম', 'বেভান', 'বোগাস', 'ব্যস্', 'ব্লো', 'ভিন্নপাঠ', 'ভূপালী', 'মহাকালে', 'মহি', 'মারাঠা', 'মুখুয্যে', 'য়', 'যাকগে', 'যাগগে', 'যায়নি', 'র', 'রাণি', 'ল', 'লম্বোদরবাবু', 'লে', 'ল্যান', 'শুনছেন', 'শ্রীরাগ', 'সঞ্জীবনী', 'সম্পর্কীয়', 'সরি', 'স্বচ্ছন্দে', 'হুপার', 'হুম', 'হুররে', 'হ্যাঁস্য', 'িছ', 'য়েছ', '০০', '

In [81]:
df=pd.read_csv("ebanglalibrary_100K_validated_with_words_bn_mark.csv");
unique_char_ipa_v2=[' ', '!', '#', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '£', '\xad', '·', 'ŋ', 'ɐ', 'ɔ', 'ɛ', 'ɟ', 'ɪ', 'ɽ', 'ɾ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', 'ː', '̃', '̆', '̪', '̯', '͡', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', '৷', '\u200c', '–', '—', '‖', '‘', '’', '“', '”', '•', '…', '∣', '\ufeff']

for row in df.itertuples():
  for ch in row.validated_ipa:
    if ch not in unique_char_ipa_v2:
      unique_char_ipa_v2.append(ch)
unique_char_ipa_v2.sort()
print(unique_char_ipa_v2)

[' ', '!', '#', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '£', '\xad', '·', 'ŋ', 'ɐ', 'ɔ', 'ɛ', 'ɟ', 'ɪ', 'ɽ', 'ɾ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', 'ː', '̃', '̆', '̪', '̯', '͡', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', '৷', '\u200c', '–', '—', '‖'

In [34]:

df_e2_50k=pd.read_csv("error_ipa2_50k.csv")
print(df_e2_50k.shape)
df_error=df_e2_50k.drop_duplicates(subset=['bn_words'], keep='first')
print(df_error.shape)
df_error.to_csv("error_ipa2_50k.csv",index=False)


(984, 6)
(963, 6)


: 